<a href="https://colab.research.google.com/github/shahdhesham/Colab-Thesis/blob/main/DeepSeek_Coder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
DeepSeek= "deepseek-ai/deepseek-coder-6.7b-instruct"

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [3]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
# Load model and tokenizer
model_name = DeepSeek
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)  # FP16 is commonly used with LLAMA models to reduce memory usage with minimal impact on performance
tokenizer.pad_token = tokenizer.eos_token  # Fix pad token warning

# Initial C code
c_code = """
#include <stdio.h>

int main() {
    int a, b;
    printf("Enter two numbers: ");
    scanf("%d %d", &a, &b);
    printf("Sum: %d\n", a + b);
    return 0;
}
"""

def translate(c_code, refinement_prompt=""):
    # Build chat messages
    user_prompt = f"""
    Translate this C code to C++ code:
    {refinement_prompt}

    C Code:
    {c_code}

    C++ Code:
    """


    messages = [
        {"role": "system", "content": "You are a code translator. Convert C code to C++ code without adding any extra text."},
        {"role": "user", "content": user_prompt}
    ]


    # Tokenize with chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True
    ).to(model.device)

    # Generate output
    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        do_sample=False,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id
    )

    # Extract and return only the C++ code
    # decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # # Clean up unwanted preamble like "assistant" or extra labels
    # cleaned = decoded.split("C++ Code:")[-1]
    # cleaned = cleaned.replace("assistant", "").strip()  # The only way to remove assistant, can't be removed from prompt
    #  return cleaned

    generated_ids = outputs[:, inputs.shape[1]:]  # Keep this critical line
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded.strip()



# Initial run
translated = translate(c_code)
print("\nInitial C++ translation:\n")
print(translated)

# Interactive loop
while True:
    refinement = input("\nRefinement prompt (or type 'quit'): ")
    if refinement.lower() == 'quit':
        break
    translated = translate(c_code, refinement)
    print("\nRefined C++ code:\n")
    print(translated)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.



Initial C++ translation:

#include <iostream>

int main() {
    int a, b;
    std::cout << "Enter two numbers: ";
    std::cin >> a >> b;
    std::cout << "Sum: " << a + b << std::endl;
    return 0;
}

Refinement prompt (or type 'quit'): quit
